In [1]:
%pwd  #look at the current work dir
%cd ~/Repos/SECT
%matplotlib inline

#dataproc
import numpy as np
import pandas as pd
import datetime

import umap
import hdbscan

#plotting
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as pyo
import plotly.graph_objs as go
from IPython.display import display, HTML

# Set notebook mode to work in offline
#from scipy.cluster.hierarchy import dendrogram, linkage

#SECT code
import utils
import TemporalClusterer
import dCollector as dC
import graphing

#utils
from pathlib import Path
import urllib.parse

[WinError 3] The system cannot find the path specified: 'C:/Users/istoffa/Repos/SECT'
C:\Users\Imrich\Repos\SECT\notebooks


ModuleNotFoundError: No module named 'utils'

In [ ]:
def mentat_query_link(val):
    template='<a href="https://mentat-hub.cesnet.cz/mentat/timeline/search?{}">ips[{}:{}]</a>'
    params={'dt_from':dt_from, 'dt_to':dt_to, 'source_addrs':'', 'source_ports':'', 'submit':'Search'}
    params['source_addrs']=str(val).lstrip('[').rstrip(']').replace('\'','')
    query = urllib.parse.urlencode(params)
    
    pages=''
    step=20
    for x in range(0,len(val),step):
        params['source_addrs']=str(val[x:x+step]).lstrip('[').rstrip(']').replace('\'','')
        query_pt = urllib.parse.urlencode(params)
        pages+='\n'+template.format(query_pt,x,min(x+step,len(val)))
    
    return template.format(query,'','')+pages                          


def date_input(analysis_date):
    lst=analysis_date.split("_")

    dt_from = (datetime.datetime.strptime(lst[0],'%Y-%m-%d')-datetime.timedelta(days=1)).isoformat(sep=' ')
    #if len(lst) == 2:
    dt_to = (datetime.datetime.strptime(lst[-1], '%Y-%m-%d')+
             datetime.timedelta(days=1, hours=23, minutes=59, seconds=59)).isoformat(sep=' ')
    #else:
    #    dt_to = (datetime.datetime(lst[0]+' 23:59:59')+datetime.timedelta(days=1)).toisoformat(sep=' ')

    dirname=lst[0]+'_'+lst[-1]

    return dt_from, dt_to, dirname


def nerd_query_link(val):
    return '<a href="https://nerd.cesnet.cz/nerd/ip/{}">NERD</a>'.format(val)

def nerd_query_link_ip(val):
    return '<a href="https://nerd.cesnet.cz/nerd/ip/{}">{}</a>'.format(val, val)


def nerd_query_link_list(val):
    template='<a href="https://nerd.cesnet.cz/nerd/ip/{}">{}</a>'
    res=''
    for x in val[:-1]:
        res += template.format(x, x)+', '
    #return template.format(val[0],val[0])
    return res + template.format(val[-1],val[-1])

In [ ]:
sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)  # more options can be specified also

pyo.init_notebook_mode()

## Arguments

In [ ]:
data_loc='./data/match/'
#data_loc='Z:\\SECT\\results\\'

analysis_date='2020-03-12'

In [ ]:
dt_from, dt_to, dirname = date_input(analysis_date)

path=data_loc+dirname+'/'

print('Reading data in...\n')
print(path)

#load data 
df, clusters, series, nerd, flows, srcip, srcport = utils.load_analysis(path)

clusters['ips_sorted']=clusters.ips.apply(lambda x: list(pd.Series(x).sort_values()))
clusters['MENTAT']=clusters.ips_sorted

### Clusters found

In [ ]:
display(clusters[['size','events','score','ips_sorted','MENTAT','tags','types','origins']].sort_values(ascending=False, by=['score','size','events']).style.format({'ips_sorted': nerd_query_link_list, 'MENTAT': mentat_query_link}))

### Correlation pattern for all clusters

In [ ]:
sns.heatmap(series)

## Selected significant clusters

In [ ]:
display(clusters.loc[flows.index,['size','events','score','ips_sorted','MENTAT','tags','types','origins']].sort_values(ascending=False, by=['score','size','events']).style.format({'ips_sorted': nerd_query_link_list, 'MENTAT': mentat_query_link}))

### Correlation pattern for selected clusters

In [ ]:
sns.heatmap(series.loc[flows.index])

## Flow data views 
* Quick reminder of cluster info
* Flow data aggregated by srcip, where ip_count and ip_top shows statistics for dstip (count of unique and most frequent values)
* Flow data aggregated by srcport --//--

In [ ]:
cluster_idx=flows.index[9] # Change here from 0-9

temp = flows[cluster_idx].reset_index(drop=True)
display(clusters.loc[cluster_idx,['size','events','ips_sorted','tags']].to_frame().T.style.format({'ips_sorted': nerd_query_link_list}))

X = srcip[cluster_idx]
X['NERD'] = srcip[cluster_idx].index

max_rows=50
with pd.option_context('display.max_rows', max_rows, 'display.min_rows', max_rows):
    display(srcip[cluster_idx].head(max_rows)[[X.columns[-1]]+list(X.columns[:-1])].style.format({'NERD': nerd_query_link})) 

In [ ]:
with pd.option_context('display.max_rows', max_rows, 'display.min_rows', max_rows):
    display(srcport[cluster_idx]) 

In [ ]:
with pd.option_context('display.max_rows', None):#, 'display.min_rows', max_rows):
    display(nerd.loc[nerd['cluster']==cluster_idx,:])

In [ ]:
max_rows=100
with pd.option_context('display.max_rows', max_rows, 'display.min_rows', max_rows):
    display(temp.sample(100))#.style.format({'srcip': lambda cluster_idx: nerd_query_link_ip(x), 'dstip': lambda x: nerd_query_link_ip(x)}))
    #display(temp.style.format({'srcip': lambda cluster_idx: nerd_query_link_list([x]), 'dstip': lambda x: nerd_query_link_list([x])}))